In [30]:
import requests
from bs4 import BeautifulSoup
import re
import operator
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import leaguedashteamstats, leaguedashteamshotlocations, commonallplayers, synergyplaytypes, leaguedashplayerstats, leaguegamefinder, playergamelogs, commonteamroster, boxscoreadvancedv3, leaguedashplayershotlocations, leaguedashptdefend, \
    leaguedashptteamdefend, shotchartdetail, synergyplaytypes
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.static import teams, players

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# event_msg_action_type_conversion = {
# 72: 'PUTBACK_LAYUP', 79: 'PULLUP_JUMP_SHOT', 2: '3PT_RUNNING_JUMP_SHOT', 103: '3PT_RUNNING_PULL', 52: 'ALLEY_OOP_DUNK', 108: 'CUTTING_DUNK_SHOT', 98: 'CUTTING_LAYUP_SHOT', 93: 'DRIVING_BANK_HOOK_SHOT', 75: 'DRIVING_FINGER_ROLL_LAYUP', 102: 'DRIVING_FLOATING_BANK_JUMP_SHOT', 101: 'DRIVING_FLOATING_JUMP_SHOT', 57: 'DRIVING_HOOK_SHOT', 6: 'DRIVING_LAYUP', 73: 'DRIVING_REVERSE_LAYUP', 63: 'FADEAWAY_JUMPER', 71: 'FINGER_ROLL_LAYUP', 78: 'FLOATING_JUMP_SHOT', 5: 'LAYUP', 44: 'REVERSE_LAYUP', 106: 'RUNNING_ALLEY_OOP_DUNK_SHOT', 50: 'RUNNING_DUNK', 76: 'RUNNING_FINGER_ROLL_LAYUP', 74: 'RUNNING_REVERSE_LAYUP', 80: 'STEP_BACK_JUMP_SHOT', 86: 'TURNAROUND_FADEAWAY', 58: 'TURNAROUND_HOOK_SHOT', 47: 'TURNAROUND_JUMP_SHOT'
# }


### Two quick functions to grab the **team_id** and the full roster with **player_ids** for a team's abbreviation ###

In [ ]:
def get_team_id(team_abbrv):
    
    # Searches nba_team list of dictionaries for team abbreviation
    
    nba_teams = teams.get_teams()
    
    print('- Grabbing team_id -')
    for team in nba_teams:
        if team['abbreviation'] == team_abbrv:
            print(f"{team['abbreviation']} team_id: {team['id']}")
            return team['id']
    
    return f'*** No team_id found for {team_abbrv} ***'

def get_roster_info(team_abbrv):
    
    team_id = get_team_id(team_abbrv)
    
    roster_finder = commonteamroster.CommonTeamRoster(
        team_id=team_id,
        season='2024-25'
    )
    
    roster = roster_finder.get_data_frames()[0]
    roster = roster[['PLAYER', 'PLAYER_ID']]
    
    
    return roster

get_roster_info('DEN')        # calling function with team abbreviation

- Grabbing team_id -
DEN team_id: 1610612743


,PLAYER,PLAYER_ID
0,Christian Braun,1631128
1,Michael Porter Jr.,1629008
2,Julian Strawther,1631124
3,Russell Westbrook,201566
4,Hunter Tyson,1641816
5,DeAndre Jordan,201599
6,Peyton Watson,1631212
7,Dario Šarić,203967
8,PJ Hall,1641790
9,DaRon Holmes II,1641747


### This code block will grab all the games played by a specified player_id in the given season, and give a generic box sxore readout. There are many other parameters that can filter this further, my favorite being **last_n_games** ###

In [32]:
playergames = playergamelogs.PlayerGameLogs(
                                            season_nullable='2024-25',
                                            # last_n_games_nullable=5,
                                            player_id_nullable=203999
                                            )

games_df = playergames.get_data_frames()[0]
print(f"Number of games: {len(games_df)}")

games_df = games_df[['GAME_DATE', 'GAME_ID', 'PLAYER_NAME', 'MATCHUP', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'PTS']]
analysis_df = pd.DataFrame(columns=games_df.columns)

variance_row = games_df.select_dtypes(include='number').var(ddof=0)
total_row = round((games_df.sum(numeric_only=True) / len(games_df)),2)

analysis_df.loc['Variance'] = round(variance_row, 2)
analysis_df.loc['Average'] = total_row

# games_df.loc['Variance'] = variance_row
# games_df.loc['Average'] = total_row

# games_df = games_df.round().astype(int)
games_df.iloc[:, 4:] = games_df.iloc[:, 4:].round().astype(int)



games_df
 
# analysis_df

Number of games: 8


,GAME_DATE,GAME_ID,PLAYER_NAME,MATCHUP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,PTS
0,2024-11-06T00:00:00,0022400177,Nikola Jokić,DEN vs. OKC,39.0,9,20,1,3,4,6,7,13,20,16,5,2,2,23
1,2024-11-04T00:00:00,0022400166,Nikola Jokić,DEN vs. TOR,38.0,10,22,1,5,7,8,5,9,14,13,7,1,2,28
2,2024-11-02T00:00:00,0022400148,Nikola Jokić,DEN vs. UTA,30.0,10,18,3,4,4,4,5,11,16,9,5,1,0,27
3,2024-11-01T00:00:00,0022400139,Nikola Jokić,DEN @ MIN,40.0,8,16,2,3,8,10,2,7,9,13,3,3,1,26
4,2024-10-29T00:00:00,0022400113,Nikola Jokić,DEN @ BKN,41.0,9,16,0,0,11,13,6,12,18,16,1,0,1,29
5,2024-10-28T00:00:00,0022400107,Nikola Jokić,DEN @ TOR,44.0,18,27,3,5,1,2,3,7,10,4,3,1,2,40
6,2024-10-26T00:00:00,0022400087,Nikola Jokić,DEN vs. LAC,37.0,14,26,7,12,6,8,3,6,9,4,5,2,1,41
7,2024-10-24T00:00:00,0022400075,Nikola Jokić,DEN vs. OKC,35.0,6,13,1,3,3,4,4,8,12,13,3,2,1,16


In [33]:
playershotlocationsfinder = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                                            season='2024-25',
                                            # last_n_games_nullable=5,
                                            team_id_nullable=1610612743
                                            )

games_df = playershotlocationsfinder.get_data_frames()[0]
games_df

SHOT_CATEGORY                                                                              Restricted Area            In The Paint (Non-RA)              Mid-Range              Left Corner 3            Right Corner 3             Above the Break 3              Backcourt             Corner 3           
columns       PLAYER_ID         PLAYER_NAME     TEAM_ID TEAM_ABBREVIATION   AGE   NICKNAME             FGM FGA FG_PCT                   FGM   FGA FG_PCT       FGM   FGA FG_PCT           FGM FGA FG_PCT            FGM  FGA FG_PCT               FGM   FGA FG_PCT       FGM  FGA FG_PCT      FGM FGA FG_PCT
0                203932        Aaron Gordon  1610612743               DEN  29.0      Aaron              20  36  0.556                   3.0   7.0  0.429       2.0   5.0  0.400             3   6  0.500            1.0  2.0   0.50               8.0  14.0  0.571       0.0  0.0    0.0        4   8  0.500
1               1631128     Christian Braun  1610612743               DEN  23.0  Christian              29  44  0.659                   3.0  11.0  0.273       1.0   3.0  0.333             4   6  0.667            0.0  5.0   0.00               9.0  17.0  0.529       0.0  0.0    0.0        4  11  0.364
2                203967         Dario Šarić  1610612743               DEN  30.0      Dario               1   5  0.200                   0.0   3.0  0.000       1.0   1.0  1.000             0   0  0.000            0.0  0.0   0.00               0.0   3.0  0.000       0.0  0.0    0.0        0   0  0.000
3                201599      DeAndre Jordan  1610612743               DEN  36.0    DeAndre               3   5  0.600                   1.0   3.0  0.333       0.0   0.0  0.000             0   0  0.000            0.0  0.0   0.00               0.0   0.0  0.000       0.0  0.0    0.0        0   0  0.000
4               1641816        Hunter Tyson  1610612743               DEN  24.0     Hunter               3   3  1.000                   0.0   1.0  0.000       1.0   1.0  1.000             0   1  0.000            0.0  1.0   0.00               0.0   3.0  0.000       0.0  0.0    0.0        0   2  0.000
5               1629618       Jalen Pickett  1610612743               DEN  25.0      Jalen               0   0  0.000                   0.0   0.0  0.000       0.0   0.0  0.000             0   0  0.000            0.0  0.0   0.00               0.0   1.0  0.000       NaN  NaN    NaN        0   0  0.000
6               1627750        Jamal Murray  1610612743               DEN  27.0      Jamal               7  12  0.583                   6.0  16.0  0.375       7.0  22.0  0.318             1   2  0.500            0.0  1.0   0.00               6.0  19.0  0.316       0.0  1.0    0.0        1   3  0.333
7               1631124    Julian Strawther  1610612743               DEN  22.0     Julian              11  19  0.579                   3.0   6.0  0.500       2.0   3.0  0.667             0   0  0.000            2.0  2.0   1.00               9.0  26.0  0.346       0.0  0.0    0.0        2   2  1.000
8               1629008  Michael Porter Jr.  1610612743               DEN  26.0    Michael              15  25  0.600                   5.0  15.0  0.333       9.0  15.0  0.600             0   1  0.000            1.0  4.0   0.25              23.0  55.0  0.418       0.0  0.0    0.0        1   5  0.200
9                203999        Nikola Jokić  1610612743               DEN  29.0     Nikola              34  56  0.607                  29.0  56.0  0.518       3.0  11.0  0.273             1   1  1.000            1.0  2.0   0.50              16.0  31.0  0.516       0.0  1.0    0.0        2   3  0.667
10              1631212       Peyton Watson  1610612743               DEN  22.0     Peyton              10  23  0.435                   2.0   9.0  0.222       1.0   3.0  0.333             1   3  0.333            1.0  2.0   0.50               1.0  11.0  0.091       0.0  0.0    0.0        2   5  0.400
11               201566   Russell Westbrook  1610612743               DEN  35.0    Russ

In [34]:
teamptdefendfinder = leaguedashptteamdefend.LeagueDashPtTeamDefend(
                                            season='2024-25',
                                            # last_n_games_nullable=5,
                                            # team_id_nullable=1610612743,
                                            # opponent_team_id_nullable=1610612743
                                            )

games_df = teamptdefendfinder.get_data_frames()[0]
games_df

,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,GP,G,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS
0,1610612760,Oklahoma City Thunder,OKC,8,8,1.0,280,699,0.401,0.450,-0.050
1,1610612744,Golden State Warriors,GSW,8,8,1.0,304,731,0.416,0.444,-0.028
2,1610612756,Phoenix Suns,PHX,8,8,1.0,320,735,0.435,0.454,-0.019
3,1610612745,Houston Rockets,HOU,8,8,1.0,302,676,0.447,0.463,-0.016
4,1610612742,Dallas Mavericks,DAL,8,8,1.0,309,706,0.438,0.453,-0.015
5,1610612765,Detroit Pistons,DET,9,9,1.0,353,769,0.459,0.469,-0.010
6,1610612748,Miami Heat,MIA,7,7,1.0,279,619,0.451,0.459,-0.009
7,1610612757,Portland Trail Blazers,POR,8,8,1.0,324,710,0.456,0.464,-0.008
8,1610612739,Cleveland Cavaliers,CLE,9,9,1.0,357,783,0.456,0.463,-0.007
9,1610612750,Minnesota Timberwolves,MIN,7,7,1.0,285,623,0.457,0.462,-0.005


In [35]:
ptdefendfinder = leaguedashptdefend.LeagueDashPtDefend(
                                            season='2024-25',
                                            # last_n_games_nullable=5,
                                            team_id_nullable=1610612743
                                            )

games_df = ptdefendfinder.get_data_frames()[0]
games_df

,CLOSE_DEF_PERSON_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,PLAYER_POSITION,AGE,GP,G,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS
0,203999,Nikola Jokić,1610612743,DEN,C,29.0,8,8,1.0,75,157,0.478,0.491,-0.014
1,1631128,Christian Braun,1610612743,DEN,G,23.0,8,8,1.0,52,107,0.486,0.460,0.026
2,1629008,Michael Porter Jr.,1610612743,DEN,F,26.0,8,8,1.0,57,106,0.538,0.481,0.057
3,1631212,Peyton Watson,1610612743,DEN,G,22.0,8,8,1.0,37,83,0.446,0.461,-0.015
4,203932,Aaron Gordon,1610612743,DEN,F,29.0,7,7,1.0,43,81,0.531,0.472,0.059
5,1627750,Jamal Murray,1610612743,DEN,G,27.0,5,5,1.0,34,76,0.447,0.463,-0.015
6,1631124,Julian Strawther,1610612743,DEN,G,22.0,8,8,1.0,29,62,0.468,0.459,0.008
7,201566,Russell Westbrook,1610612743,DEN,G,35.0,8,8,1.0,23,54,0.426,0.444,-0.018
8,1641816,Hunter Tyson,1610612743,DEN,F,24.0,4,4,1.0,8,30,0.267,0.431,-0.164
9,203967,Dario Šarić,1610612743,DEN,F-C,30.0,5,5,1.0,16,30,0.533,0.479,0.054


In [36]:
shotchartdetailfinder = shotchartdetail.ShotChartDetail(
                                            season_nullable='2024-25',
                                            # last_n_games_nullable=5,
                                            # team_id_nullable=1610612743
                                            player_id=201566,
                                            team_id=1610612743
                                            )

games_df = shotchartdetailfinder.get_data_frames()[0]
games_df

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022400075,191,201566,Russell Westbrook,1610612743,Denver Nuggets,2,11,51,Made Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,23,11,1,1,20241024,DEN,OKC
1,Shot Chart Detail,0022400075,621,201566,Russell Westbrook,1610612743,Denver Nuggets,4,7,18,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-233,-6,1,1,20241024,DEN,OKC
2,Shot Chart Detail,0022400107,214,201566,Russell Westbrook,1610612743,Denver Nuggets,2,8,40,Made Shot,Running Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,20,21,1,1,20241028,TOR,DEN
3,Shot Chart Detail,0022400107,517,201566,Russell Westbrook,1610612743,Denver Nuggets,4,10,51,Made Shot,Turnaround Fadeaway shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,7,11,71,1,1,20241028,TOR,DEN
4,Shot Chart Detail,0022400107,665,201566,Russell Westbrook,1610612743,Denver Nuggets,4,0,24,Made Shot,Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-9,0,1,1,20241028,TOR,DEN
5,Shot Chart Detail,0022400113,106,201566,Russell Westbrook,1610612743,Denver Nuggets,1,3,10,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,12,24,1,1,20241029,BKN,DEN
6,Shot Chart Detail,0022400113,480,201566,Russell Westbrook,1610612743,Denver Nuggets,3,2,50,Made Shot,Fadeaway Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,12,-60,109,1,1,20241029,BKN,DEN
7,Shot Chart Detail,0022400113,515,201566,Russell Westbrook,1610612743,Denver Nuggets,3,0,4,Made Shot,Step Back Jump shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,25,104,231,1,1,20241029,BKN,DEN
8,Shot Chart Detail,0022400113,523,201566,Russell Westbrook,1610612743,Denver Nuggets,4,11,14,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,18,20,1,1,20241029,BKN,DEN
9,Shot Chart Detail,0022400113,541,201566,Russell Westbrook,1610612743,Denver Nuggets,4,9,55,Made Shot,Running Pull-Up Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,25,-165,188,1,1,20241029,BKN,DEN
